In [1]:
import folium
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy.exc import GeocoderTimedOut
import pandas as pd


Defined function that take two lists of addresses and return distance and geographic coordinates (like latitude and longitude)


In [2]:
def Distance_coordinate_addresses(list_address1, list_address2):
    
    if not list_address1 or not list_address2:
        raise ValueError("Entrambe le liste devono essere popolate da almeno un indirizzo")
    
    if len(list_address1) != len(list_address2):
        raise ValueError("Le due liste sono di dimensione differente")
    

    geolocator = Nominatim(user_agent= "My-Agent", timeout= 10)
    distance = []
    coordinate = []

    for i , (indirizzo1, indirizzo2) in enumerate(zip(list_address1, list_address2), start=1):
        if not indirizzo1:
            raise ValueError(f"L'indirizzo in posizione {i} della prima lista è vuoto")
        location_ind1 = geolocator.geocode(indirizzo1)
        if location_ind1 is None:
            raise ValueError(f"L'indirizzo {indirizzo1} in posizione {i} della prima lista non è valido o non è stato trovato")
        
        ##### stessa cosa per gli indirizzi della seconda lista
        if not indirizzo2:
            raise ValueError(f"L'indirizzo in posizione {i} della seconda lista è vuoto")
        location_ind2 = geolocator.geocode(indirizzo2)
        if location_ind2 is None:
            raise ValueError(f"L'indirizzo {indirizzo2} in posizione {i} della seconda lista non è valido o non è stato trovato")
        
        coordinate_1 = (location_ind1.latitude, location_ind1.longitude)
        coordinate_2 = (location_ind2.latitude, location_ind2.longitude)
        distanze = geodesic(coordinate_1, coordinate_2).kilometers
        distance.append(distanze)
        coordinate.append((coordinate_1, coordinate_2))

    
    return distance, coordinate

In [9]:
l1 = ['Piazza San Marco, Venezia', 'Oxford Street, London']
l2 = ['Via Marsala, Roma', 'Via Nino Bixio, Genova']


In [ ]:
try:
    distances, coordinates = Distance_coordinate_addresses(l1,l2)
    for i, (distance, (c1, c2)) in enumerate(zip(distances, coordinates), start=1):
        print(f"Distanza tra gli indirizzi {i}: {distance:.2f} km")
        print(f"Coordinate Indirizzo {i} - Lista 1: {c1}")
        print(f"Coordinate Indirizzo {i} - Lista 2: {c2}")
except ValueError as e:
    print(e)


Defined the map center based on geographic coordinates

In [11]:
def Centro_mappa(coordinates):
    latitudes = [coord[0] for pair in coordinates for coord in pair]
    longitudes = [coord[1] for pair in coordinates for coord in pair]
    
    center_lat = sum(latitudes) / len(latitudes)
    center_lon = sum(longitudes) / len(longitudes)
    
    return [center_lat, center_lon]

The map returns the distance in beeline between pairs of addresses. 

In [14]:
def Mappa_distanze_geo(coordinate):

    map_center = Centro_mappa(coordinate)
    

    my_map = folium.Map(location=map_center, zoom_start=4)
    
    # Aggiungi i marker e la linea per ogni coppia di coordinate
    for coord1, coord2 in coordinate:
        # Aggiungi i marker per i due punti
        folium.Marker(location=coord1, popup="Indirizzo 1", icon=folium.Icon(color='blue')).add_to(my_map)
        folium.Marker(location=coord2, popup="Indirizzo 2", icon=folium.Icon(color='red')).add_to(my_map)
        
        # Aggiungi una linea tra i due punti
        folium.PolyLine(locations=[coord1, coord2], color="black", weight=2.5, opacity=0.5).add_to(my_map)
    
    # Salva la mappa come file HTML
    nome_mappa = "mappa_distanze_tra_indirizzi.html"
    my_map.save(nome_mappa)
    print(f"Mappa salvata come {nome_mappa}")
    
    return nome_mappa

In [ ]:
Mappa_distanze_geo(coordinates)

In [ ]:
import os
print(f"File salvato in: {os.getcwd()}")